In [1]:
require("prepare_data_Sioux_simp.jl")
require("demands_adjustment_Sioux_simp_gradi")
include("tap_Sioux_simp.jl")

TAP (generic function with 1 method)

In [2]:
# load Jacobian
jacobian = readall("jacobian.json");
jacobian = JSON.parse(jacobian);

jacob = zeros(numODpairs, numLinks)
for i = 1:numLinks
    for j = 1:numODpairs
        jacob[j, i] = jacobian[i][j]
    end
end

In [3]:
jacob

132x30 Array{Float64,2}:
 -0.0   0.0   0.0  0.0   0.0  -0.0  0.0  …   1.0  -0.0  0.0   0.0  0.0   1.0
  0.0   0.0   0.0  0.0   0.0   0.0  0.0      0.0   0.0  0.0   0.0  0.0   0.0
  0.0  -1.0   0.0  0.0   0.0   0.0  0.0      1.0   0.0  0.0   1.0  0.0   0.0
 -1.0   0.0   0.0  0.0   0.0   0.0  0.0      0.0   0.0  0.0   0.0  0.0   0.0
  0.0   0.0   0.0  0.0   0.0   1.0  0.0      0.0   1.0  0.0   0.0  0.0   0.0
  0.0   0.0   0.0  0.0   0.0   0.0  0.0  …   1.0   0.0  0.0   0.0  0.0   0.0
  0.0   0.0  -0.0  0.0   0.0   0.0  0.0      0.0   0.0  0.0  -0.0  0.0   0.0
 -1.0   0.0   1.0  0.0   0.0   0.0  0.0      0.0   1.0  0.0   0.0  0.0   0.0
  0.0   0.0   0.0  0.0   0.0   0.0  0.0      0.0   0.0  0.0   0.0  0.0   0.0
  0.0   0.0   0.0  0.0   0.0   0.0  0.0      0.0   0.0  0.0   0.0  0.0   0.0
 -1.0   0.0   0.0  0.0   0.0   0.0  0.0  …   0.0   0.0  0.0   0.0  0.0   1.0
  0.0   0.0   1.0  0.0   0.0   0.0  0.0      0.0   0.0  0.0   0.0  0.0   0.0
  0.0   0.0   0.0  0.0  -0.0   0.0  0.0     -0.0   

In [4]:
x, x_0

([2070.0,3510.0,2070.0,3150.0,3510.0,6240.91,2849.09,6284.98,9810.0,3900.91  …  7110.0,4410.0,12150.0,12150.0,7110.0,3854.98,7200.0,3885.02,2805.02,3929.09],[2300.0,3900.0,2300.0,3500.0,3900.0,7060.55,3039.45,7105.27,10650.3,4202.51  …  7900.0,5149.72,13758.0,13749.7,8158.04,4155.55,8249.72,4194.73,2994.73,4239.45])

In [5]:
# compute the gradient
gradi_ = zeros(numODpairs)
for i = 1:numODpairs
    gradi_[i] = sum([2 * (x[j] - x_0[j]) * jacob[i, j] for j = 1:numLinks])
end

In [6]:
numNodes

12

In [7]:
OD_pair_label_dict = Dict()
OD_pair_label_dict_ = Dict()
label = 1
for i = 1:numNodes
    for j = 1:numNodes
        if i != j
            key = "($(i), $(j))"
            OD_pair_label_dict[key] = label
            OD_pair_label_dict_[label] = key
            label += 1
        end
    end
end

In [8]:
OD_pair_label_dict_[numODpairs]

"(12, 11)"

In [9]:
#load OD pair labels
odPairLabel = readall("od_pair_label_dict_Sioux_simp_refined.json");
odPairLabel = JSON.parse(odPairLabel);

odPairLabel_ = readall("od_pair_label_dict__Sioux_simp_refined.json");
odPairLabel_ = JSON.parse(odPairLabel_);

In [10]:
# ensure the consistency of dimensions

gradi = zeros(numODpairs)

for i = 1:numODpairs
    if ~("($(odPairLabel_["$(i)"][1]), $(odPairLabel_["$(i)"][2]))" in keys(OD_pair_label_dict))
        gradi[i] = 0
    else
        gradi[i] = gradi_[OD_pair_label_dict["($(odPairLabel_["$(i)"][1]), $(odPairLabel_["$(i)"][2]))"]]
    end
end

In [11]:
length(demands)

144

In [12]:
# for convenience of computing the new demand vector
demandsVec = zeros(length(odPairLabel_))

for i = 1:length(demandsVec)
    demandsVec[i] = demands[(odPairLabel_["$i"][1], odPairLabel_["$i"][2])]
end

In [13]:
demandsVec

132-element Array{Float64,1}:
  100.0
  100.0
  500.0
  200.0
  300.0
  500.0
  800.0
  500.0
 1300.0
  500.0
  200.0
  100.0
  100.0
    ⋮  
 1400.0
  200.0
  100.0
  200.0
  600.0
  200.0
  200.0
  700.0
  600.0
  600.0
 2000.0
 1400.0

In [14]:
demands

Dict{(Int64,Int64),Float64} with 144 entries:
  (8,10)  => 1600.0
  (9,5)   => 800.0
  (3,6)   => 300.0
  (10,5)  => 1000.0
  (6,9)   => 400.0
  (12,4)  => 600.0
  (4,4)   => 0.0
  (9,4)   => 700.0
  (10,11) => 4000.0
  (8,9)   => 800.0
  (9,1)   => 500.0
  (11,1)  => 500.0
  (8,11)  => 800.0
  (3,1)   => 100.0
  (1,10)  => 1300.0
  (1,12)  => 200.0
  (12,7)  => 700.0
  (4,5)   => 500.0
  (9,9)   => 0.0
  (11,8)  => 800.0
  (12,8)  => 600.0
  (2,4)   => 200.0
  (6,5)   => 200.0
  (4,9)   => 700.0
  (8,4)   => 700.0
  ⋮       => ⋮

In [15]:
# convert the demand data into dictionary; for convenience of solving the TAP problem and the inverse VI problem

demandsDict = Dict{(Int64,Int64), Float64}()

for key = [(int(split(key, ", ")[1][2:end]), int(split(key, ", ")[2][1:end-1])) for key = keys(odPairLabel)]
    demandsDict[key] = demandsVec[odPairLabel["($(key[1]), $(key[2]))"]]
end

In [16]:
demandsDict

Dict{(Int64,Int64),Float64} with 132 entries:
  (3,8)   => 200.0
  (9,5)   => 800.0
  (3,6)   => 300.0
  (10,5)  => 1000.0
  (6,9)   => 400.0
  (12,4)  => 600.0
  (11,1)  => 500.0
  (9,4)   => 700.0
  (10,11) => 4000.0
  (8,9)   => 800.0
  (9,1)   => 500.0
  (8,11)  => 800.0
  (3,1)   => 100.0
  (1,10)  => 1300.0
  (1,12)  => 200.0
  (12,7)  => 700.0
  (4,5)   => 500.0
  (11,8)  => 800.0
  (12,8)  => 600.0
  (2,4)   => 200.0
  (6,5)   => 200.0
  (4,9)   => 700.0
  (8,4)   => 700.0
  (9,2)   => 200.0
  (1,2)   => 100.0
  ⋮       => ⋮

In [17]:
gradi

132-element Array{Float64,1}:
  -4212.86      
  -6880.0       
  -4296.04      
    460.0       
 -11983.8       
  -3396.07      
      0.0       
  -3845.07      
  -3479.45      
     -0.0       
  -3404.65      
  -3400.59      
      0.0       
      ⋮         
  -3621.27      
     -0.0       
  -3359.96      
     -0.0       
  -1919.41      
  -8198.69      
  -2096.07      
      2.46018e-9
  -2700.43      
  -8755.48      
  -4640.72      
  -6483.37      

In [18]:
# compute a descent direction

r = similar(gradi)
for i = 1:length(gradi)
    r[i] = -1 * gradi[i]
end

In [19]:
r

132-element Array{Float64,1}:
  4212.86      
  6880.0       
  4296.04      
  -460.0       
 11983.8       
  3396.07      
    -0.0       
  3845.07      
  3479.45      
     0.0       
  3404.65      
  3400.59      
    -0.0       
     ⋮         
  3621.27      
     0.0       
  3359.96      
     0.0       
  1919.41      
  8198.69      
  2096.07      
    -2.46018e-9
  2700.43      
  8755.48      
  4640.72      
  6483.37      

In [20]:
# compute a search direction

r_ = similar(r)
for i = 1:length(r)
    if (demandsVec[i] > 20) || (demandsVec[i] <= 20 && r[i] > 0)
        r_[i] = r[i]
    else
        r_[i] = 0
    end
end

In [21]:
r_

132-element Array{Float64,1}:
  4212.86      
  6880.0       
  4296.04      
  -460.0       
 11983.8       
  3396.07      
    -0.0       
  3845.07      
  3479.45      
     0.0       
  3404.65      
  3400.59      
    -0.0       
     ⋮         
  3621.27      
     0.0       
  3359.96      
     0.0       
  1919.41      
  8198.69      
  2096.07      
    -2.46018e-9
  2700.43      
  8755.48      
  4640.72      
  6483.37      

In [22]:
# line search

alphaList = Float64[]

for i = 1:length(r_)
    if r_[i] < 0
        push!(alphaList, -demandsVec[i]/r_[i])
    end
end

alpha_max = minimum(alphaList)

0.33918950375054446

In [23]:
length(r_)

132

In [24]:
# update

demandsVecNew = similar(demandsVec)
for i = 1:length(demandsVec)
    demandsVecNew[i] = demandsVec[i] + (alpha_max / 81) * r_[i] 
end

In [25]:
demandsVecNew

132-element Array{Float64,1}:
  117.641
  128.81 
  517.99 
  198.074
  350.182
  514.221
  800.0  
  516.101
 1314.57 
  500.0  
  214.257
  114.24 
  100.0  
    ⋮    
 1415.16 
  200.0  
  114.07 
  200.0  
  608.038
  234.332
  208.777
  700.0  
  611.308
  636.664
 2019.43 
 1427.15 

In [26]:
# convert the demand data into dictionary; for convenience of solving the TAP problem and the inverse VI problem

demandsDictNew = Dict{(Int64,Int64), Float64}()

for key = [(int(split(key, ", ")[1][2:end]), int(split(key, ", ")[2][1:end-1])) for key = keys(odPairLabel)]
    demandsDictNew[key] = demandsVecNew[odPairLabel["($(key[1]), $(key[2]))"]]
end

for i=1:numNodes
    key = (i,i)
    demandsDictNew[key] = 0
end

In [27]:
demandsDictNew

Dict{(Int64,Int64),Float64} with 144 entries:
  (3,8)   => 210.2011110559348
  (9,5)   => 808.6261473007821
  (3,6)   => 318.442244402921
  (10,5)  => 1009.4639595675221
  (6,9)   => 405.35987800475436
  (12,4)  => 608.0375725045335
  (4,4)   => 0.0
  (11,1)  => 497.1524831787349
  (9,4)   => 701.9262613799931
  (10,11) => 4016.847291161965
  (8,9)   => 800.0
  (9,1)   => 500.0
  (8,11)  => 838.1744625587004
  (3,1)   => 105.9602594398481
  (1,10)  => 1314.570264755785
  (1,12)  => 214.2570409956566
  (12,7)  => 699.9999999999897
  (4,5)   => 509.7203863089406
  (11,8)  => 802.9312673161477
  (9,9)   => 0.0
  (12,8)  => 611.308105529695
  (2,4)   => 213.23257816992265
  (6,5)   => 229.2944351246492
  (4,9)   => 726.4688501825659
  (8,4)   => 706.6162890860734
  ⋮       => ⋮

In [30]:
tapFlowVect = TAP(demands)

30-element Array{Float64,1}:
  2300.0 
  3900.0 
  2300.0 
  3500.0 
  3900.0 
  7060.55
  3039.45
  7105.27
 10650.3 
  4202.51
 10642.0 
  8050.28
  7900.0 
     ⋮   
  6500.0 
  5158.04
  7900.0 
  5149.72
 13758.0 
 13749.7 
  8158.04
  4155.55
  8249.72
  4194.73
  2994.73
  4239.45

In [31]:
demands

Dict{(Int64,Int64),Float64} with 144 entries:
  (8,10)  => 1600.0
  (9,5)   => 800.0
  (3,6)   => 300.0
  (10,5)  => 1000.0
  (6,9)   => 400.0
  (12,4)  => 600.0
  (4,4)   => 0.0
  (9,4)   => 700.0
  (10,11) => 4000.0
  (8,9)   => 800.0
  (9,1)   => 500.0
  (11,1)  => 500.0
  (8,11)  => 800.0
  (3,1)   => 100.0
  (1,10)  => 1300.0
  (1,12)  => 200.0
  (12,7)  => 700.0
  (4,5)   => 500.0
  (9,9)   => 0.0
  (11,8)  => 800.0
  (12,8)  => 600.0
  (2,4)   => 200.0
  (6,5)   => 200.0
  (4,9)   => 700.0
  (8,4)   => 700.0
  ⋮       => ⋮